In [54]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp

Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 43.2MB/s]


In [55]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [56]:
df=pd.read_csv('preprocessed_data.csv')

In [57]:
df.head(4)

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [58]:
def preprocess(x):
  x=x[:-1]
  return x

In [59]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [60]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [61]:
df.shape

(2000, 2)

In [62]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [63]:
df=df[df['source'].apply(length)<170]
df=df[df['target'].apply(length)<200]

In [64]:
df.shape

(1990, 2)

In [65]:
df['target_in'] = '<start> ' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + ' <end>'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [66]:
df=df.drop('target',axis=1)


In [67]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>


In [68]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)

In [69]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+' <end>'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+' <end>'

(1970, 3) (20, 3)


In [70]:
tknizer_source = Tokenizer()
tknizer_source.fit_on_texts(train['source'].values)
tknizer_target = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_target.fit_on_texts(train['target_in'].values)

In [71]:
vocab_size_target=len(tknizer_target.word_index.keys())
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())
print(vocab_size_source)

3029
3698


In [72]:
tknizer_target.word_index['<start>'], tknizer_target.word_index['<end>']

(1, 1439)

In [73]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True,name="embedding_layer_encoder")
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):

      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer oupulstm_state_h,lstm_state_ct to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      
      input_embedd                           = self.embedding(input_sequence)
      lstm_state_h,lstm_state_c= states[0],states[1]
      self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
      return self.lstm_output,lstm_state_h,lstm_state_c


    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [tf.zeros((batch_size,self.lstm_size)),tf.zeros((batch_size,self.lstm_size))]
      


In [74]:
#https://github.com/UdiBhaskar/TfKeras-Custom-Layers/blob/master/Seq2Seq/clayers.py
def _attention_score(dec_ht,
                     enc_hs,
                     attention_type,
                     weightwa=None,
                     weightua=None,
                     weightva=None):
    if attention_type == 'bahdanau':
        score = weightva(tf.nn.tanh(weightwa(dec_ht) + weightua(enc_hs)))
        score = tf.squeeze(score, [2])
    elif attention_type == 'dot':
        score = tf.matmul(dec_ht, enc_hs, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'general':
        score = weightwa(enc_hs)
        score = tf.matmul(dec_ht, score, transpose_b=True)
        score = tf.squeeze(score, 1)
    elif attention_type == 'concat':
        dec_ht = tf.tile(dec_ht, [1, enc_hs.shape[1], 1])
        score = weightva(tf.nn.tanh(weightwa(tf.concat((dec_ht, enc_hs), axis=-1))))
        score = tf.squeeze(score, 2)
        
    return score

In [75]:
def _monotonic_attetion(probabilities, attention_prev, mode):

    """Compute monotonic attention distribution from choosing probabilities.
    Implemented Based on -
    https://colinraffel.com/blog/online-and-linear-time-attention-by-enforcing-monotonic-alignments.html
    https://arxiv.org/pdf/1704.00784.pdf
    Mainly implemented by referring
    https://github.com/craffel/mad/blob/b3687a70615044359c8acc440e43a5e23dc58309/example_decoder.py#L22
    # Arguments:
        probabilities: Probability of choosing input sequence..
                       Should be of shape (batch_size, max_length),
                       and should all be in the range [0, 1].
        attention_prev: The attention distribution from the previous output timestep.
                            Should be of shape (batch_size, max_length).
                            For the first output timestep,
                            should be [1, 0, 0, ...,0] for all n in [0, ... batch_size - 1].
        mode: How to compute the attention distribution.
              Must be one of 'recursive', 'parallel', or 'hard'.
              - 'recursive' uses tf.scan to recursively compute the distribution.
              This is slowest but is exact, general, and does not suffer from
              numerical instabilities.
              - 'parallel' uses parallelized cumulative-sum and cumulative-product
              operations to compute a closed-form solution to the recurrence relation
              defining the attention distribution.  This makes it more efficient than 'recursive',
              but it requires numerical checks which make the distribution non-exact.
              This can be a problem in particular when max_length is long and/or
              probabilities has entries very close to 0 or 1.
              - 'hard' requires that  the probabilities in p_choose_i are all either 0 or 1,
              and subsequently uses a more efficient and exact solution.
    # Returns: A tensor of shape (batch_size, max_length) representing the attention distributions
               for each sequence in the batch.
    # Raises:
             ValueError: if mode is not one of 'recursive', 'parallel', 'hard'."""
    if mode == 'hard':
        #Remove any probabilities before the index chosen last time step
        probabilities = probabilities*tf.cumsum(attention_prev, axis=1)
        attention = probabilities*tf.math.cumprod(1-probabilities, axis=1, exclusive=True)
    elif mode == 'recursive':
        batch_size = tf.shape(input=probabilities)[0]
        shifted_1mp_probabilities = tf.concat([tf.ones((batch_size, 1)),\
            1 - probabilities[:, :-1]], 1)
        attention = probabilities*tf.transpose(a=tf.scan(lambda x, yz: tf.reshape(yz[0]*x + yz[1],\
            (batch_size,)), [tf.transpose(a=shifted_1mp_probabilities),\
                tf.transpose(a=attention_prev)], tf.zeros((batch_size,))))
    elif mode == 'parallel':
        cumprod_1mp_probabilities = tf.exp(tf.cumsum(tf.math.log(tf.clip_by_value(1-probabilities,\
            1e-10, 1)), axis=1, exclusive=True))
        attention = probabilities*cumprod_1mp_probabilities*tf.cumsum(attention_prev/\
            tf.clip_by_value(cumprod_1mp_probabilities, 1e-10, 1.), axis=1)
    else:
        raise ValueError("Mode must be 'hard', 'parallel' or 'recursive' ")

    return attention 

In [76]:
class MonotonicBahdanauAttention(tf.keras.layers.Layer):
    '''
    MonotonicBahdanauAttention
    Implemented based on below paper
        https://arxiv.org/pdf/1704.00784.pdf
    # Arguments
        units = number of hidden units to use.
        mode = How to compute the attention distribution.
              Must be one of 'recursive', 'parallel', or 'hard'.
              - 'recursive' uses tf.scan to recursively compute the distribution.
              This is slowest but is exact, general, and does not suffer from
              numerical instabilities.
              - 'parallel' uses parallelized cumulative-sum and cumulative-product
              operations to compute a closed-form solution to the recurrence relation
              defining the attention distribution.  This makes it more efficient than 'recursive',
              but it requires numerical checks which make the distribution non-exact.
              This can be a problem in particular when max_length is long and/or
              probabilities has entries very close to 0 or 1.
              - 'hard' requires that  the probabilities in p_choose_i are all either 0 or 1,
              and subsequently uses a more efficient and exact solution.
        return_aweights = Bool, whether to return attention weights or not.
        scaling_factor = int/float to scale the score vector. default None=1
        noise_std = standard deviation of noise which will be added before
                    applying sigmoid function.(pre-sigmoid noise). If it is 0 or
                    mode="hard", we won't add any noise.
        weights_initializer = initializer for weight matrix
        weights_regularizer = Regularize the weights
        weights_constraint = Constraint function applied to the weights
    # Returns
        context_vector = context vector after applying attention.
        attention_weights = attention weights only if `return_aweights=True`.
    # Inputs to the layer
        inputs = dictionary with keys "enocderHs", "decoderHt", "prevAttention".
                enocderHs = all the encoder hidden states,
                            shape - (Batchsize, encoder_seq_len, enc_hidden_size)
                decoderHt = hidden state of decoder at that timestep,
                            shape - (Batchsize, dec_hidden_size)
                prevAttention = Previous probability distribution of attention
                                (previous attention weights)
        mask = You can apply mask for padded values or any custom values
               while calculating attention.
               if you are giving mask for encoder and deocoder then you have
               to give a dict similar to inputs. (keys: enocderHs, decoderHt)
               else you can give only for enocoder normally.(one tensor)
               mask shape should be (Batchsize, encoder_seq_len)
    '''
 
    def __init__(self, units,
                 mode='parallel',
                 return_aweights=False,
                 scaling_factor=None,
                 noise_std=0,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',
                 weights_regularizer=None,
                 bias_regularizer=None,
                 weights_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'name' not in kwargs:
            kwargs['name'] = ""
        super(MonotonicBahdanauAttention, self).__init__(**kwargs)
        self.units = units
        self.mode = mode
        self.return_aweights = return_aweights
        self.scaling_factor = scaling_factor
        self.noise_std = noise_std
        self.weights_initializer = tf.keras.initializers.get(weights_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.weights_regularizer = tf.keras.regularizers.get(weights_regularizer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.weights_constraint = tf.keras.constraints.get(weights_constraint)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)
        self._wa = tf.keras.layers.Dense(self.units, use_bias=False,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Wa")
        self._ua = tf.keras.layers.Dense(self.units,\
            kernel_initializer=weights_initializer, bias_initializer=bias_initializer,\
                kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                    kernel_constraint=weights_constraint, bias_constraint=bias_constraint,\
                        name=self.name+"Ua")
        self._va = tf.keras.layers.Dense(1, use_bias=False, kernel_initializer=weights_initializer,\
            kernel_regularizer=weights_regularizer, bias_regularizer=bias_regularizer,\
                bias_initializer=bias_initializer, kernel_constraint=weights_constraint,\
                    bias_constraint=bias_constraint, name=self.name+"Va")
        self.supports_masking = True


    def call(self, inputs, training=True):
        '''call'''
        #assert isinstance(inputs, dict)
       
 
        if ('enocderHs' not in inputs.keys())or ('decoderHt' not in inputs.keys()\
            or 'prevAttention' not in inputs.keys()):
            raise ValueError("Input to the layer must be a dict with \
            keys=['enocderHs','decoderHt', 'prevAttention']")
 
        #if isinstance(mask, dict):
        #   mask_enc = mask.get('enocderHs', None)
        #   mask_dec = mask.get('decoderHt', None)
        #else:
        #    mask_enc = mask
        #    mask_dec = None
        enc_out, dec_prev_hs = tf.cast(inputs['enocderHs'], tf.float32), \
            tf.cast(inputs['decoderHt'], tf.float32)
 
        prev_attention = inputs['prevAttention']
      
 
        #if mask_dec is not None:
           # dec_prev_hs = dec_prev_hs * tf.cast(mask_dec, dec_prev_hs.dtype)
        #if mask_enc is not None:
           # enc_out = enc_out * tf.cast(tf.expand_dims(mask_enc, 2), enc_out.dtype)
 
        # decprev_hs - Decoder hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        dec_hidden_with_time_axis = tf.expand_dims(dec_prev_hs, 1)
       
 
        # score shape == (batch_size, max_length)
        score = _attention_score(dec_ht=dec_hidden_with_time_axis, enc_hs=enc_out,\
                    attention_type='concat', weightwa=self._wa,\
                        weightua=self._ua, weightva=self._va)
 
        if self.scaling_factor is not None:
            score = score/tf.sqrt(self.scaling_factor)
 
        if training:
            if self.noise_std > 0:
                random_noise = tf.random.normal(shape=tf.shape(input=score), mean=0,\
                    stddev=self.noise_std, dtype=score.dtype, seed=self.seed)
                score = score + random_noise
 
        #if mask_enc is not None:
        #    score = score + (tf.cast(tf.math.equal(mask_enc, False), score.dtype)*-1e9)
 
        if self.mode == 'hard':
            probabilities = tf.cast(score > 0, score.dtype)
        else:
            probabilities = tf.sigmoid(score)
 
        attention_weights = _monotonic_attetion(probabilities, prev_attention, self.mode)
        attention_weights = tf.expand_dims(attention_weights, 1)
 
        #context_vector shape (batch_size, hidden_size)
        context_vector = tf.matmul(attention_weights, enc_out)
        context_vector = tf.squeeze(context_vector, 1, name="context_vector")
 
        if self.return_aweights:
         
            return context_vector, tf.squeeze(attention_weights, 1, name='attention_weights')
        return context_vector

In [77]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,mode ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length 
      self.dec_units = dec_units
      self.mode = mode
      self.att_units = att_units
      # we are using embedding_matrix and not training the embedding layer
      self.embedding = tf.keras.layers.Embedding(input_dim=self.tar_vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True,name="embedding_layer_decoder")
      self.lstm = tf.keras.layers.LSTM(self.dec_units, return_sequences=True, return_state=True)
      self.dense = tf.keras.layers.Dense(self.tar_vocab_size)
      self.attention = MonotonicBahdanauAttention(self.att_units,mode=self.mode,return_aweights=True)

  def call(self,input_to_decoder, encoder_output, state_h,state_c,attention_weights):

   
    inputs= dict()
    inputs['enocderHs']=encoder_output
    inputs['decoderHt']=state_h
    inputs['prevAttention']=attention_weights

    output = self.embedding(input_to_decoder)
    context_vector,attention_weights = self.attention(inputs)
    context_vector1 = tf.expand_dims(context_vector,1)
    concat = tf.concat([output,context_vector1],axis=-1)
    decoder_output,state_h,state_c = self.lstm(concat,initial_state=[state_h,state_c])
    final_output = self.dense(decoder_output)
    final_output = tf.reshape(final_output,(-1,final_output.shape[2]))
    return final_output,state_h,state_c,attention_weights,context_vector




In [78]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,mode ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super(Decoder,self).__init__()
      self.vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units=dec_units
      self.att_units=att_units
      self.mode=mode
      self.onestepdecoder=One_Step_Decoder(self.vocab_size,self.embedding_dim,self.input_length,self.dec_units ,self.mode ,self.att_units)
        
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state,attention_weights ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        all_outputs=tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1])
        #Create a tensor array as shown in the reference notebook
        #Iterate till the length of the decoder input
        for timestep in range(tf.shape(input_to_decoder)[1]):
            # Call onestepdecoder for each token in decoder_input
            output,state_h,state_c,attention_weights,context_vector=self.onestepdecoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state,attention_weights)

            # Store the output in tensorarray
            all_outputs=all_outputs.write(timestep,output)
        all_outputs=tf.transpose(all_outputs.stack(),[1,0,2])
        # Return the tensor array
        return all_outputs
        
        
    

In [79]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size,mode):
    #Intialize objects from encoder decoder
    super().__init__() # https://stackoverflow.com/a/27134600/4084039
    self.batch_size=batch_size
    self.encoder_inputs_length=encoder_inputs_length
    self.decoder_inputs_length=decoder_inputs_length
    self.encoder = Encoder(vocab_size_source+1,300,128,self.encoder_inputs_length)
    self.decoder = Decoder(vocab_size_target+1,300,self.decoder_inputs_length,128,mode,128)
    
  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    input,output = data[0], data[1]
    attention_weights = np.zeros((self.batch_size,self.encoder_inputs_length),dtype='float32')
    attention_weights[:,0]=1
    initial_state=self.encoder.initialize_states(self.batch_size)
    encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
    decoder_output= self.decoder(output, encoder_output, encoder_h, encoder_c,attention_weights)
    return decoder_output




In [80]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [81]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [82]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,39,39)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,39,39)

train_dataloader = Dataloder(train_dataset, batch_size=20)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 39) (512, 39) (512, 39)


In [83]:
tf.config.experimental_run_functions_eagerly(True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [84]:
tf.config.run_functions_eagerly(True)


In [91]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
# Implement teacher forcing while training your model. You can do it two ways.
# Prepare your data, encoder_input,decoder_input and decoder_output
# if decoder input is 
# <start> Hi how are you
# decoder output should be
# Hi How are you <end>
# i.e when you have send <start>-- decoder predicted Hi, 'Hi' decoder predicted 'How' .. e.t.c

# or
 
# model.fit([train_ita,train_eng],train_eng[:,1:]..)
# Note: If you follow this approach some grader functions might return false and this is fine.
model  = encoder_decoder(encoder_inputs_length=39,decoder_inputs_length=39,output_vocab_size=vocab_size_target,batch_size=512,mode="parallel")
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss=loss_function)
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=260)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/260
3/3 [==============================] - 2s 743ms/step - loss: 3.0825
Epoch 2/260
3/3 [==============================] - 2s 765ms/step - loss: 2.5597
Epoch 3/260
3/3 [==============================] - 2s 792ms/step - loss: 2.4022
Epoch 4/260
3/3 [==============================] - 2s 729ms/step - loss: 2.3590
Epoch 5/260
3/3 [==============================] - 2s 732ms/step - loss: 2.3331
Epoch 6/260
3/3 [==============================] - 2s 721ms/step - loss: 2.3062
Epoch 7/260
3/3 [==============================] - 2s 722ms/step - loss: 2.2742
Epoch 8/260
3/3 [==============================] - 2s 744ms/step - loss: 2.2484
Epoch 9/260
3/3 [==============================] - 2s 718ms/step - loss: 2.2178
Epoch 10/260
3/3 [==============================] - 2s 726ms/step - loss: 2.1843
Epoch 11/260
3/3 [==============================] - 2s 749ms/step - loss: 2.1481
Epoch 12/260
3/3 [==============================] - 2s 796ms/step - loss: 2.1150
Epoch 13/260
3/3 [===================

In [92]:
batch_size=512
units=128

In [93]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to onestepdecoder.
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predictions, input_states, attention_weights = model.layers[1].onestepdecoder(input_to_decoder, encoder_output, input_states)
         Save the attention weights
         And get the word using the tokenizer(word index) and then store it in a string.
  E. Call plot_attention(#params)
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=39)
  attention_weights = np.zeros((1,39),dtype='float32')
  attention_weights[:,0]=1

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['<start>']]])
  pred = []
  #Here 43 is the max_length of the sequence
  for i in range(39):
    output,state_h,state_c,attention_weights,context_vector = model.layers[1].onestepdecoder(cur_vec,en_outputs,state_h,state_c,attention_weights)
    cur_vec = np.reshape(np.argmax(output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='<end>'):
      break
    translated_sentence = ' '.join(pred)
  return translated_sentence

In [94]:
for i in validation['source']:
  print("The Actual output is: ",i)
  predicted=predict(i)
  print("The predicted output is: ",predicted)
  print(">"*180)


The Actual output is:  Wow, haha, can go try b mistress 4 few days lor, get a taste of it... Aiya, in army one lor, u wont noe one.... Den mei one is kua's fren, thgt u noe liao...
The predicted output is:  i just got time do you that i don't know what time then where are quite well hope you are you mum there maybe it's okay all depends on tuesday i've got like to see die we go and
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual output is:  Same as u 1245... I oso wan shop, but cant leh, parents dun let me go out liao...
The predicted output is:  ok i'm sorry to watch in camp too so how about so bored or smu for your give my like there and then end of her outside this show is there then there will tell you get it then
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [95]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [96]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 1000 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 1000 test data sentences is:  0.19536971776642903


In [97]:
bleu_scores_lst


[0.4001601601922499,
 0,
 0,
 0.4001601601922499,
 0.47587330964125224,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.47587330964125224,
 0.4001601601922499,
 0,
 0.47897362544357464,
 0.4001601601922499,
 0.4001601601922499,
 0.47587330964125224,
 0]